# Principle component analysis projection (MADlib v1.10+)
Demonstrates PCA projection including grouping, which was introduced in MADlib v1.10

In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
%sql postgresql://gpdbchina@10.194.10.68:55000/madlib
#%sql postgresql://fmcquillan@localhost:5432/madlib
#%sql postgresql://gpadmin@54.197.30.46:10432/gpadmin

u'Connected: gpdbchina@madlib'

In [3]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.10.0-dev, git revision: rc/v1.9alpha-rc1-91-ga0325d6, cmake configuration time: Wed Jan 11 01:36:53 UTC 2017, build type: Release, build system: Linux-2.6.18-238.27.1.el5.hotfix.bz516490, C compiler: gcc 4.4.0, C++ compiler: g++ 4.4.0"


# 1.  Create sample data in dense matrix form

In [58]:
%%sql 
DROP TABLE IF EXISTS mat;

CREATE TABLE mat (
    id integer,
    row_vec double precision[]
);

INSERT INTO mat VALUES
(1, '{1,2,3}'),
(2, '{2,1,2}'),
(3, '{3,2,1}');

SELECT * FROM mat ORDER BY id;

Done.
Done.
3 rows affected.
3 rows affected.


id,row_vec
1,"[1.0, 2.0, 3.0]"
2,"[2.0, 1.0, 2.0]"
3,"[3.0, 2.0, 1.0]"


# 2.  Run the PCA function for a specified number of principal components

In [59]:
%%sql
DROP TABLE IF EXISTS result_table, result_table_mean;

SELECT madlib.pca_train('mat',             -- Source table
                        'result_table',    -- Output table
                        'id',              -- Row id of source table
                         2);               -- Number of principle components

SELECT * FROM result_table ORDER BY row_id;

Done.
1 rows affected.
2 rows affected.


row_id,principal_components,std_dev,proportion
1,"[0.707106781186547, -4.16333634234434e-17, -0.707106781186547]",1.41421356237,0.857142857142
2,"[-2.77555756156289e-17, -1.0, -4.16333634234434e-17]",0.57735026919,0.142857142857


# 3.  Projection in dense matrix form
Project the original data to a lower dimensional representation, and view the result of the projection:

In [60]:
%%sql
DROP TABLE IF EXISTS residual_table, result_summary_table, out_table;
SELECT madlib.pca_project( 'mat',
                            'result_table',
                            'out_table',
                            'id',
                            'residual_table',
                            'result_summary_table'
                            );

Done.
1 rows affected.
3 rows affected.


row_id,row_vec
1,"[-1.41421356237309, -0.33333333333]"
2,"[2.77555756157677e-17, 0.66666666667]"
3,"[1.41421356237309, -0.33333333333]"


Check the error in the projection:

In [17]:
%%sql
SELECT * FROM result_summary_table;

1 rows affected.


exec_time,residual_norm,relative_residual_norm
344.725847244,3.92699236582,0.645593568096


Check the residuals:

In [18]:
%%sql
SELECT * FROM residual_table ORDER BY row_id;

3 rows affected.


row_id,row_vec
1,"[0.0253387670918417, 0.346033388731942, -1.18096378457015]"
2,"[0.0397264537778188, 0.542515718038531, -1.85153062226574]"
3,"[0.0656887038551082, 0.897063567221092, -3.06155307505916]"


# 4.  Grouping with dense matrices
Create sample data in dense matrix form with a grouping column:

In [61]:
%%sql 
DROP TABLE IF EXISTS mat_group;

CREATE TABLE mat_group (
    id integer,
    row_vec double precision[],
    matrix_id integer
);

INSERT INTO mat_group VALUES
(1, '{1,2,3}', 1),
(2, '{2,1,2}', 1),
(3, '{3,2,1}', 1),
(4, '{1,2,3,4,5}' ,2),
(5, '{2,5,2,4,1}' ,2),
(6, '{5,4,3,2,1}' ,2);

SELECT * FROM mat_group ORDER BY id;

Done.
Done.
6 rows affected.
6 rows affected.


id,row_vec,matrix_id
1,"[1.0, 2.0, 3.0]",1
2,"[2.0, 1.0, 2.0]",1
3,"[3.0, 2.0, 1.0]",1
4,"[1.0, 2.0, 3.0, 4.0, 5.0]",2
5,"[2.0, 5.0, 2.0, 4.0, 1.0]",2
6,"[5.0, 4.0, 3.0, 2.0, 1.0]",2


# 5.  Run the PCA function with grouping for a specified proportion of variance

In [62]:
%%sql
DROP TABLE IF EXISTS result_table_group, result_table_group_mean;

SELECT madlib.pca_train('mat_group',             -- Source table
                        'result_table_group',    -- Output table
                        'id',                    -- Row id of source table
                         0.8,                    -- Proportion of variance
                        'matrix_id');            -- Grouping column

SELECT * FROM result_table_group ORDER BY matrix_id, row_id;

Done.
1 rows affected.
3 rows affected.


row_id,principal_components,std_dev,proportion,matrix_id
1,"[-0.707106781186548, -1.11022302462516e-16, 0.707106781186548]",1.41421356237,0.857142857142,1
1,"[0.555378486712784, 0.388303582074091, -0.0442457354870797, -0.255566375612852, -0.688115693174023]",3.23152203117,0.764102534485,2
2,"[0.587384101786277, -0.485138064894743, 0.311532046315153, -0.449458074050715, 0.347212037159181]",1.79553112719,0.235897465516,2


# 6. Projection in dense matrix form with grouping 
Run the PCA project function on subsets of an input table based on grouping columns. Note that the parameter 'pc_table' used for projection must also be generated in training using the same grouping columns. 

In [63]:
%%sql 
DROP TABLE IF EXISTS mat_group_projected;
SELECT madlib.pca_project('mat_group',
                          'result_table_group',
                          'mat_group_projected',
                          'id');
SELECT * FROM mat_group_projected ORDER BY matrix_id, row_id;

Done.
1 rows affected.
6 rows affected.


row_id,row_vec,matrix_id
1,[1.4142135623731],1
2,[7.40148683087139e-17],1
3,[-1.4142135623731],1
4,"[-3.59290479201926, 0.559694003674779]",2
5,"[0.924092949098971, -2.00871628417505]",2
6,"[2.66881184290186, 1.44902228049511]",2


# 6.  Sparse matrices
Create sample data in sparse matrix form: 

In [64]:
%%sql
DROP TABLE IF EXISTS mat_sparse;

CREATE TABLE mat_sparse (
    row_id integer,
    col_id integer,
    value double precision
);

INSERT INTO mat_sparse VALUES
(1, 1, 1.0),
(2, 2, 2.0),
(3, 3, 3.0),
(4, 4, 4.0),
(1, 5, 5.0),
(2, 4, 6.0),
(3, 2, 7.0),
(4, 3, 8.0);

SELECT * FROM mat_sparse ORDER BY row_id, col_id;

Done.
Done.
8 rows affected.
8 rows affected.


row_id,col_id,value
1,1,1.0
1,5,5.0
2,2,2.0
2,4,6.0
3,2,7.0
3,3,3.0
4,3,8.0
4,4,4.0


This is what the sparse matrix looks like when put in dense form: 

In [65]:
%%sql
DROP TABLE IF EXISTS mat_dense;

SELECT madlib.matrix_densify('mat_sparse', 
                            'row=row_id, col=col_id, val=value', 
                            'mat_dense');

SELECT * FROM mat_dense ORDER BY row_id;

Done.
1 rows affected.
4 rows affected.


row_id,value
1,"[1.0, 0.0, 0.0, 0.0, 5.0]"
2,"[0.0, 2.0, 0.0, 6.0, 0.0]"
3,"[0.0, 7.0, 3.0, 0.0, 0.0]"
4,"[0.0, 0.0, 8.0, 4.0, 0.0]"


# 7.  Run the PCA sparse function for a specified number of principal components

In [66]:
%%sql
DROP TABLE IF EXISTS result_table, result_table_mean;

SELECT madlib.pca_sparse_train( 'mat_sparse',       -- Source table
                                'result_table',     -- Output table
                                'row_id',           -- Row id of source table
                                'col_id',           -- Column id of source table
                                'value',            -- Value of matrix at row_id, col_id
                                4,                  -- Actual number of rows in the matrix
                                5,                  -- Actual number of columns in the matrix
                                3);                 -- Number of principle components
                               
SELECT * FROM result_table ORDER BY row_id;

Done.
1 rows affected.
3 rows affected.


row_id,principal_components,std_dev,proportion
1,"[0.0876046030186159, 0.0968983772909993, -0.798223884258748, -0.392314423461549, 0.438023015093079]",4.2136280383,0.436590030617
2,"[-0.0647272661608605, 0.877639526308691, 0.0872109579144957, -0.336471420895091, -0.323636330804302]",3.68408023747,0.333748701545
3,"[0.0780380267884856, -0.177956517174911, 0.550872495873674, -0.711711105037595, 0.390190133942428]",3.0560690806,0.229661267838


# 8.  Projection in sparse matrix form
Project the original sparse data to low-dimensional representation:

In [67]:
%%sql
DROP TABLE IF EXISTS mat_sparse_out;

SELECT madlib.pca_sparse_project(
                    'mat_sparse',
                    'result_table',
                    'mat_sparse_out',
                    'row_id',
                    'col_id',
                    'value',
                    4,
                    5
                    );

SELECT * FROM mat_sparse_out ORDER BY row_id;

Done.
1 rows affected.
4 rows affected.


row_id,row_vec
1,"[4.66617015032369, -2.63552220635847, 2.1865220849604]"
2,"[0.228360685652383, -1.21616275892926, -4.46864627611561]"
3,"[0.672067460100428, 5.45249627172823, 0.56445525585642]"
4,"[-5.5665982960765, -1.6008113064405, 1.71766893529879]"


# 8. Grouping with sparse matrices
Create sample data in sparse matrix form:

In [68]:
%%sql
DROP TABLE IF EXISTS mat_sparse_group;

CREATE TABLE mat_sparse_group (
    row_id integer,
    col_id integer,
    value double precision,
    matrix_id integer);

INSERT INTO mat_sparse_group VALUES
(1, 1, 1.0, 1),
(2, 2, 2.0, 1),
(3, 3, 3.0, 1),
(4, 4, 4.0, 1),
(1, 5, 5.0, 1),
(2, 4, 6.0, 2),
(3, 2, 7.0, 2),
(4, 3, 8.0, 2);

SELECT * FROM mat_sparse_group ORDER BY matrix_id, row_id, col_id;

Done.
Done.
8 rows affected.
8 rows affected.


row_id,col_id,value,matrix_id
1,1,1.0,1
1,5,5.0,1
2,2,2.0,1
3,3,3.0,1
4,4,4.0,1
2,4,6.0,2
3,2,7.0,2
4,3,8.0,2


Run the PCA function with grouping for a specified proportion of variance

In [69]:
%%sql
DROP TABLE IF EXISTS result_table_group, result_table_group_mean;

SELECT madlib.pca_sparse_train( 'mat_sparse_group',       -- Source table
                                'result_table_group',     -- Output table
                                'row_id',           -- Row id of source table
                                'col_id',           -- Column id of source table
                                'value',            -- Value of matrix at row_id, col_id
                                4,                 -- Actual number of rows in the matrix
                                5,                 -- Actual number of columns in the matrix
                                0.8,                 -- Proportion of variance
                                'matrix_id');
                               
SELECT * FROM result_table_group ORDER BY matrix_id, row_id;

Done.
1 rows affected.
4 rows affected.


row_id,principal_components,std_dev,proportion,matrix_id
1,"[-0.17805696611353, 0.0681313257646982, 0.140140679619953, 0.389110973431455, -0.890284830567652]",2.73659933166,0.544652792875,1
2,"[-0.0492086814863994, 0.149371585357526, 0.523758898591052, -0.800253642890858, -0.246043407431997]",2.06058314533,0.308800210824,1
1,"[0.0, -0.479486114660443, 0.860935716335885, -0.169949281213684, 0.0]",4.40325305088,0.520500333693,2
2,"[0.0, 0.689230898585949, 0.249586247854404, -0.680196643123418, 0.0]",3.74355664586,0.376220573443,2


# 9. Projection in sparse format with grouping

In [70]:
%%sql
DROP TABLE IF EXISTS mat_sparse_group_projected;
SELECT madlib.pca_sparse_project(
    'mat_sparse_group',
    'result_table_group',
    'mat_sparse_group_projected',
    'row_id',
    'col_id',
    'value',
    4,
    5
    );

SELECT * FROM mat_sparse_group_projected ORDER BY matrix_id, row_id;

Done.
1 rows affected.
8 rows affected.


row_id,row_vec,matrix_id
1,"[-4.00039298524261, -0.626820612715982]",1
2,"[0.765350785238575, 0.951348276645455]",1
3,"[1.04951017256904, 2.22388180170356]",1
4,"[2.185532027435, -2.54840946563303]",1
1,"[-0.627846810195469, -0.685031603549092]",2
2,"[-1.64754249747757, -4.7662114622896]",2
3,"[-3.98424961281857, 4.13958468655255]",2
4,"[6.25963892049161, 1.31165837928614]",2
